FMCG Sales Data for Time Series Forecasting and Decision Analysis

# Problem Background
 In the fast-paced world of Fast-Moving Consumer Goods (FMCG), understanding sales dynamics can be the difference between a product flying off the shelves or gathering dust. 
 This dataset offers a glimpse into the intricate dance of sales volume, pricing, promotions, and more across various store locations.

# Problem Definition
Use the data to predict sales volume based on other features provided.


# Data 
  https://www.kaggle.com/datasets/krishanukalita/fmcg-sales-demand-forecasting-and-optimization/data




In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Import ml libraries
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import KFold 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score

from sklearn.tree import export_graphviz

In [6]:
df = pd.read_csv('extended_fmcg_demand_forecasting.csv', parse_dates=['Date'])
df.head()

,Date,Product_Category,Sales_Volume,Price,Promotion,Store_Location,Weekday,Supplier_Cost,Replenishment_Lead_Time,Stock_Level
0,2022-01-01,Household,1583,5.190661,0,Urban,5,9.299281,9,207
1,2022-01-02,Personal Care,1103,8.949596,0,Urban,6,13.274109,5,253
2,2022-01-03,Dairy,455,4.867987,0,Rural,0,13.302265,9,245
3,2022-01-04,Personal Care,1107,16.968596,1,Urban,1,10.056158,5,265
4,2022-01-05,Personal Care,1447,4.309673,1,Rural,2,3.562862,8,334


In [7]:
df.columns

Index(['Date', 'Product_Category', 'Sales_Volume', 'Price', 'Promotion',
       'Store_Location', 'Weekday', 'Supplier_Cost', 'Replenishment_Lead_Time',
       'Stock_Level'],
      dtype='object')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Date                     1000 non-null   datetime64[ns]
 1   Product_Category         1000 non-null   object        
 2   Sales_Volume             1000 non-null   int64         
 3   Price                    1000 non-null   float64       
 4   Promotion                1000 non-null   int64         
 5   Store_Location           1000 non-null   object        
 6   Weekday                  1000 non-null   int64         
 7   Supplier_Cost            1000 non-null   float64       
 8   Replenishment_Lead_Time  1000 non-null   int64         
 9   Stock_Level              1000 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(5), object(2)
memory usage: 78.2+ KB


##### Checking for missing values if any

In [9]:
df.isna().sum()

Date                       0
Product_Category           0
Sales_Volume               0
Price                      0
Promotion                  0
Store_Location             0
Weekday                    0
Supplier_Cost              0
Replenishment_Lead_Time    0
Stock_Level                0
dtype: int64

### Feature Engineering

In [10]:
data = df.copy()

In [14]:
data['Year'] = data.Date.dt.year
data['Month'] = data.Date.dt.month
data.head()

,Date,Product_Category,Sales_Volume,Price,Promotion,Store_Location,Weekday,Supplier_Cost,Replenishment_Lead_Time,Stock_Level,Year,Month
0,2022-01-01,Household,1583,5.190661,0,Urban,5,9.299281,9,207,2022,1
1,2022-01-02,Personal Care,1103,8.949596,0,Urban,6,13.274109,5,253,2022,1
2,2022-01-03,Dairy,455,4.867987,0,Rural,0,13.302265,9,245,2022,1
3,2022-01-04,Personal Care,1107,16.968596,1,Urban,1,10.056158,5,265,2022,1
4,2022-01-05,Personal Care,1447,4.309673,1,Rural,2,3.562862,8,334,2022,1


In [15]:
# Dropping date column
data.drop("Date", axis = 1, inplace=True)
data.head()

,Product_Category,Sales_Volume,Price,Promotion,Store_Location,Weekday,Supplier_Cost,Replenishment_Lead_Time,Stock_Level,Year,Month
0,Household,1583,5.190661,0,Urban,5,9.299281,9,207,2022,1
1,Personal Care,1103,8.949596,0,Urban,6,13.274109,5,253,2022,1
2,Dairy,455,4.867987,0,Rural,0,13.302265,9,245,2022,1
3,Personal Care,1107,16.968596,1,Urban,1,10.056158,5,265,2022,1
4,Personal Care,1447,4.309673,1,Rural,2,3.562862,8,334,2022,1


### Data Preprocessing
 - Converting Categorical variables to numerical

In [16]:
data.select_dtypes(["object"])

,Product_Category,Store_Location
0,Household,Urban
1,Personal Care,Urban
2,Dairy,Rural
3,Personal Care,Urban
4,Personal Care,Rural
...,...,...
995,Snacks,Suburban
996,Beverages,Urban
997,Beverages,Suburban
998,Household,Suburban


In [17]:
data['Product_Category'].value_counts()

Beverages        210
Household        206
Personal Care    204
Dairy            190
Snacks           190
Name: Product_Category, dtype: int64

In [18]:
data['Store_Location'].value_counts()


Urban       342
Rural       338
Suburban    320
Name: Store_Location, dtype: int64

In [20]:
data['Product_Category_cat'] = data['Product_Category'].astype('category').cat.codes
data['Store_Location_cat'] = data['Store_Location'].astype('category').cat.codes
data.head()

,Product_Category,Sales_Volume,Price,Promotion,Store_Location,Weekday,Supplier_Cost,Replenishment_Lead_Time,Stock_Level,Year,Month,Product_Category_cat,Store_Location_cat
0,Household,1583,5.190661,0,Urban,5,9.299281,9,207,2022,1,2,2
1,Personal Care,1103,8.949596,0,Urban,6,13.274109,5,253,2022,1,3,2
2,Dairy,455,4.867987,0,Rural,0,13.302265,9,245,2022,1,1,0
3,Personal Care,1107,16.968596,1,Urban,1,10.056158,5,265,2022,1,3,2
4,Personal Care,1447,4.309673,1,Rural,2,3.562862,8,334,2022,1,3,0


In [21]:
data.drop(['Product_Category','Store_Location'], axis = 1, inplace=True)

In [22]:
data.head()

,Sales_Volume,Price,Promotion,Weekday,Supplier_Cost,Replenishment_Lead_Time,Stock_Level,Year,Month,Product_Category_cat,Store_Location_cat
0,1583,5.190661,0,5,9.299281,9,207,2022,1,2,2
1,1103,8.949596,0,6,13.274109,5,253,2022,1,3,2
2,455,4.867987,0,0,13.302265,9,245,2022,1,1,0
3,1107,16.968596,1,1,10.056158,5,265,2022,1,3,2
4,1447,4.309673,1,2,3.562862,8,334,2022,1,3,0


### Modelling
- Splitting dataset

In [24]:
X = data.drop('Sales_Volume', axis = 1)
y = data['Sales_Volume']

In [26]:
# Slitting the data (70% for training and 30% for testing).
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

y_test_actual = y_test


- Scaling data

In [28]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Train the Model